In [17]:
import pandas as pd


df = pd.read_csv('C:/Users/Admin/PycharmProjects/analytics/online_retail_analysis/data/online_retail.csv', encoding='windows-1251')

# df.head()
# Общая информация по данным
# df.info()

# Посчитает пропуски по каждой колонке
# df.isna().sum()

# 1. Первичная очистка данных

# Приводим дату к datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Убираем строки без CustomerID
df = df.dropna(subset=['CustomerID'])

# Оставляем только реальные покупки
df = df[df['Quantity'] > 0]

# добавляем значение выручки
df['Revenue'] = df['Quantity']*df['UnitPrice']

df.info()

## Задача №1: Очистка данных

## В рамках подготовки данных:
## - колонка InvoiceDate приведена к типу datetime
## - удалены строки без CustomerID
## - исключены возвраты и отменённые заказы
## - рассчитана выручка по каждой позиции

df.head()

# Шаг 1 - отделяем возвраты

returns = df[df['Quantity'] < 0]
sales = df[df['Quantity'] > 0]

returns.shape, sales.shape

# Данные о возвратах отсутствуют — после фильтрации датафрейм оказался пустым.
# Основной датасет продаж содержит 397 924 записи и 9 признаков, что позволяет проводить статистический и поведенческий анализ.

<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      397924 non-null  object        
 8   Revenue      397924 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 30.4+ MB


((0, 9), (397924, 9))